## Reporter Import of selected products

In [15]:
import pandas as pd
import datetime
import time

In [16]:
# Enter years, separated by '%2C'
years = '2016%2C2017%2C2018'
cty_code = 'all'

In [29]:
# Import of the selected product codes
products_df = pd.read_excel("Importer_of_Selected_Products_code.xlsx", dtype = {'codes':str})
products_dict = products_df.set_index('product group').T.to_dict('list')
products_dict

{'Textiles': ['SITC', 'S4', '65', 'N'],
 'Toys, Dolls & Games': ['SITC', 'S4', '8942, 8943, 8944', 'N'],
 'Complete Watches': ['SITC', 'S4', '8853, 8854', 'Y'],
 'Assembled Watch Movements': ['SITC', 'S4', '8855', 'Y'],
 'Diamond, Cut, Not Mounted': ['SITC', 'S4', '66729', 'N'],
 'Travel Goods & Handbags': ['SITC', 'S4', '831', 'N'],
 'Clothing': ['SITC', 'S4', '84', 'N'],
 'Footwear': ['SITC', 'S4', '851', 'N'],
 'Wine': ['HS', 'H4', '2204, 2205', 'N'],
 'Spirit': ['HS', 'H4', '2208', 'N'],
 'Fur Clothing': ['SITC', 'S4', '84831', 'N'],
 'Spectacles & Frames': ['SITC', 'S4', '88421, 88423', 'N']}

In [30]:
# As UNcomtrade API use '%2C' string to separate codings
# modify the coding string here
for k, v in products_dict.items():
    v[2] = v[2].replace(', ', '%2C')
    print(v[2])

65
8942%2C8943%2C8944
8853%2C8854
8855
66729
831
84
851
2204%2C2205
2208
84831
88421%2C88423


In [31]:
products_dict

{'Textiles': ['SITC', 'S4', '65', 'N'],
 'Toys, Dolls & Games': ['SITC', 'S4', '8942%2C8943%2C8944', 'N'],
 'Complete Watches': ['SITC', 'S4', '8853%2C8854', 'Y'],
 'Assembled Watch Movements': ['SITC', 'S4', '8855', 'Y'],
 'Diamond, Cut, Not Mounted': ['SITC', 'S4', '66729', 'N'],
 'Travel Goods & Handbags': ['SITC', 'S4', '831', 'N'],
 'Clothing': ['SITC', 'S4', '84', 'N'],
 'Footwear': ['SITC', 'S4', '851', 'N'],
 'Wine': ['HS', 'H4', '2204%2C2205', 'N'],
 'Spirit': ['HS', 'H4', '2208', 'N'],
 'Fur Clothing': ['SITC', 'S4', '84831', 'N'],
 'Spectacles & Frames': ['SITC', 'S4', '88421%2C88423', 'N']}

In [32]:
# implement function for downloading data from UNcomtrade website
def download_data(productsdict=products_dict,periods=years,countrycode=cty_code):
    
    append_dict={}
    for k, v in productsdict.items():
        
        codetype=v[1]
        
        UNcomtrade_link = 'http://comtrade.un.org/api/get?max=5000000&type=C&freq=A&'\
                            f'px={codetype}&'\
                            f'ps={years}&'\
                            f'r={countrycode}&'\
                            'p=0&rg=all&'\
                            f'cc={v[2]}&'\
                            'fmt=csv&head=M'
        #print(UNcomtrade_link)
    
        df = pd.read_csv(UNcomtrade_link, dtype={'cmdCode': str})
        
        
        append_dict[k]=df
        
    return append_dict

In [33]:
data_dict = download_data(productsdict=products_dict,periods=years,countrycode=cty_code)

In [34]:
data_dict

{'Textiles':      pfCode    yr  period  periodDesc  aggrLevel  IsLeaf  rgCode     rgDesc  \
 0        S4  2016    2016        2016          2       0       1     Import   
 1        S4  2016    2016        2016          2       0       2     Export   
 2        S4  2016    2016        2016          2       0       1     Import   
 3        S4  2016    2016        2016          2       0       2     Export   
 4        S4  2016    2016        2016          2       0       1     Import   
 5        S4  2016    2016        2016          2       0       2     Export   
 6        S4  2016    2016        2016          2       0       4  Re-Import   
 7        S4  2016    2016        2016          2       0       1     Import   
 8        S4  2016    2016        2016          2       0       2     Export   
 9        S4  2016    2016        2016          2       0       1     Import   
 10       S4  2016    2016        2016          2       0       2     Export   
 11       S4  2016    2016  

In [35]:
# export raw data to excel
writer = pd.ExcelWriter('import_industry_rawdata.xlsx', engine='xlsxwriter')
for key, values in data_dict.items(): 
    
    values.to_excel(writer, sheet_name=key)
writer.save()
    

In [36]:
# implement function to perform sorting for country ranking
def trade_ranking(data, tradetype='Import',quantity='N'): # data is dataframe format
    
    tradedata = data[data['rgDesc'] == tradetype]
    
    # create HK mil column
    tradedata.loc[:,'TradeValue_HKD_Million'] = tradedata['TradeValue'].values*7.8/10**6
    # change column names
    tradedata = tradedata.rename(columns={'TradeValue': 'TradeValue_USD_Dollar'})
    
    # create quantity in thousand column
    if quantity=='Y':
        tradedata.loc[:,'quantity(thousand)'] = tradedata['qtAltCode'].values/1000
    
    tradedata_result = tradedata.groupby(['rtTitle','yr','rgDesc']).sum().unstack('yr')
    latest_yr = tradedata_result.columns.levels[1][-1]
    
    if quantity=='Y':
        tradedata_result = tradedata_result[['TradeValue_USD_Dollar','TradeValue_HKD_Million','qtAltCode','quantity(thousand)']]
    else:
        tradedata_result = tradedata_result[['TradeValue_USD_Dollar','TradeValue_HKD_Million']]
        
    tradedata_result.sort_values([('TradeValue_HKD_Million', latest_yr)], ascending=False, inplace = True)
    return tradedata_result

In [37]:
def trade_result(datadict, tradetype='Import'):
    append_dict={}
    for k,v in datadict.items(): # v is dataframe for each selected product category
        append_dict[k]=trade_ranking(v,tradetype='Import',quantity=products_dict[k][3])
    return append_dict

In [38]:
result = trade_result(data_dict, tradetype='Import')

In [39]:
# have a look on one selected product category
result['Complete Watches']

TradeValue_USD_Dollar                \
yr                                            2016          2017   
rtTitle               rgDesc                                       
China, Hong Kong SAR  Import          6.202345e+09  6.291909e+09   
USA                   Import          4.451797e+09  4.207764e+09   
China                 Import          1.850323e+09  2.197453e+09   
France                Import          2.593362e+09  2.640983e+09   
Japan                 Import          2.456883e+09  2.434644e+09   
Switzerland           Import          2.056189e+09  1.991741e+09   
Germany               Import          2.113217e+09  2.126530e+09   
United Kingdom        Import          1.767229e+09  1.767219e+09   
Singapore             Import          1.505484e+09  1.550118e+09   
United Arab Emirates  Import          1.104711e+09  1.660117e+09   
Italy                 Import          1.551789e+09  1.591865e+09   
Rep. of Korea         Import          6.115763e+08  6.473224e+08   
China, Macao SAR      Import          6.330873e+08           NaN   
Spain                 Import          7.874127e+08  7.473836e+08   
Netherlands           Import          5.803827e+08  5.993246e+08   
Saudi Arabia          Import          5.081820e+08  4.697837e+08   
Australia             Import          4.793475e+08  4.668892e+08   
Thailand              Import          5.000059e+08  3.949855e+08   
Other Asia, nes       Import          4.703601e+08  4.034789e+08   
Malaysia              Import          3.785169e+08  4.108906e+08   
Canada                Import          4.190818e+08  3.933771e+08   
Austria               Import          4.848944e+08  4.730867e+08   
Mexico                Import          3.084984e+08  3.007884e+08   
Russian Federation    Import          2.334049e+08  2.496185e+08   
India                 Import          1.931622e+08  2.217238e+08   
Portugal              Import          2.094826e+08  2.321067e+08   
Turkey                Import          2.200337e+08  2.380190e+08   
Qatar                 Import          2.193929e+08  2.171392e+08   
Kuwait                Import          2.586879e+08  2.440832e+08   
Poland                Import          1.452381e+08  1.652707e+08   
...                                            ...           ...   
Sao Tome and Principe Import          9.260000e+02  3.917000e+03   
Benin                 Import          1.368200e+04  3.643900e+04   
Comoros               Import          4.249000e+03  3.318000e+03   
Gambia                Import          2.201000e+03  1.042000e+03   
Algeria               Import          8.546043e+06  6.003826e+06   
Cameroon              Import          1.412100e+05  9.941000e+04   
Central African Rep.  Import          1.561300e+04           NaN   
Congo                 Import          6.248860e+05  4.733000e+05   
Dominican Rep.        Import          1.012969e+07  1.091306e+07   
Ethiopia              Import          4.077667e+06  3.759582e+06   
Guatemala             Import          1.336403e+07  1.204548e+07   
Honduras              Import          3.260005e+06  4.638873e+06   
Iran                  Import          4.635483e+07  6.587341e+07   
Jamaica               Import          1.275485e+07  1.621137e+07   
Kiribati              Import          1.115600e+04           NaN   
Lesotho               Import                   NaN  1.633900e+05   
Malawi                Import          5.752600e+04  9.246800e+04   
Mali                  Import          2.893770e+05  2.277760e+05   
Mauritania            Import          7.248800e+04  2.933600e+04   
Nepal                 Import          5.091570e+06  4.550138e+06   
Niger                 Import          3.322400e+04           NaN   
Panama                Import          7.821120e+07           NaN   
Rwanda                Import          2.181380e+05           NaN   
Saint Kitts and Nevis Import          3.891438e+06  4.044471e+06   
Saint Lucia           Import          2.748350e+05  3.558230e+05   
Sierra Leone     

In [40]:
# export the result to excel format
writer = pd.ExcelWriter('Import_result.xlsx', engine='xlsxwriter')
for key, values in result.items(): 
    values.to_excel(writer, sheet_name=key, freeze_panes=(2,0))
writer.save()